In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
from datetime import datetime

In [2]:
# https://lol.fandom.com/wiki/LEC/2022_Season/Spring_Season/Match_History

In [3]:
math_list = r"https://gol.gg/tournament/tournament-matchlist/CBLOL%20Split%201%202023/"

### Request match data

In [4]:
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.110 Safari/537.36"}
html = requests.get("https://gol.gg/game/stats/44921/page-summary/", headers=headers).content
soup = BeautifulSoup(html, 'html.parser')

In [5]:
# print(soup.prettify())

# How find divs in parser
# https://linuxpip.org/beautifulsoup-find-an-element-by-class/

### Name of teams in the match

In [6]:
#Team blue side
blue_team = soup.find("div", class_="row rowbreak fond-main-cadre p-4").find_all("div", class_="col-4 col-sm-5 text-center")[0].get_text()
blue_result = soup.find("div", class_="row rowbreak fond-main-cadre p-4").find_all("div", class_="col-4 col-sm-5 text-center")[2].find("h1").get_text()

#Team red side
red_team = soup.find("div", class_="row rowbreak fond-main-cadre p-4").find_all("div", class_="col-4 col-sm-5 text-center")[1].get_text()
red_result = soup.find("div", class_="row rowbreak fond-main-cadre p-4").find_all("div", class_="col-4 col-sm-5 text-center")[3].find("h1").get_text()

### Duration of the match (minutes, seconds)

In [7]:
# Match time
match_time = datetime.strptime(soup.find("div", class_="col-4 col-sm-2 text-center align-middle right-side-win").find_all("h1")[0].get_text(), '%M:%S').time()
match_time

datetime.time(0, 35, 9)

### Champions Banned and Picked

In [8]:
#First 5 is the banned champions for blue side
#Last 5 picke champions for blue side
blue_champs = soup.find("div", class_="row rowbreak fond-main-cadre p-4").find_all("div", class_="col-4 col-sm-5 text-center")[2].find_all("img")

#Same for the red side
red_champs = soup.find("div", class_="row rowbreak fond-main-cadre p-4").find_all("div", class_="col-4 col-sm-5 text-center")[3].find_all("img")

In [9]:
lst_blue_champs = []
for champ in blue_champs:
    lst_blue_champs.append(champ["src"].split('/')[-1].split('.')[0])

lst_red_champs = []
for champ in red_champs:
    lst_red_champs.append(champ["src"].split('/')[-1].split('.')[0])

In [10]:
# picks and bans of blue side
blue_banned = lst_blue_champs[:5]
blue_piked = lst_blue_champs[5:]

In [11]:
#picks and bans of red side
red_banned = lst_red_champs[:5]
red_piked = lst_red_champs[5:]

### Team Stats (Kills, Towers, Dragons and Barons)

In [12]:
#Ordem of prints, Kills, Towers, Dragons, Barons

#list of stats of each team
blue_stats = []
red_stats = []
#For blue team

for i in soup.find_all("div", class_="col-12 col-sm-6 pb-4")[0].find_all("span", class_="score-box"):
    #Keep only numbers in the string
    blue_stats.append(int(''.join(filter(str.isdigit, i.get_text()))))

#For red team
for i in soup.find_all("div", class_="col-12 col-sm-6 pb-4")[1].find_all("span", class_="score-box"):
    #Keep only numbers in the string
    red_stats.append(int(''.join(filter(str.isdigit, i.get_text()))))

In [13]:
def get_team_stats(stats, team):
    dct_team = {}
    dct_team[f"kills_{team}"] = stats[0]
    dct_team[f"towers_{team}"] = stats[1]
    dct_team[f"dragons_{team}"] = stats[2]
    dct_team[f"barons_{team}"] = stats[3]

    return dct_team

print(blue_stats)
print(red_stats)


[11, 4, 3, 0]
[19, 10, 2, 1]


### KDA of teams

In [14]:
#Find all tables and get the first(blue side)
# soup.find_all('table')[1]

In [15]:
def player_kda(row):
    """
    row: line with data with kills, deaths and assists of each player

    return the KDA rounded to 2 decimals
    """
    kills, deaths, other = row.split('/')
    assists = other.split(' ')[0]
    if int(deaths) > 0:
        kda = (int(kills) + int(assists))/int(deaths)
    else:
        kda = (int(kills) + int(assists))
        
    return round(kda, 2)

In [16]:
def get_team_kda(team):

    if team == "blue":
        idx = 0
    else:
        idx = 1
        

    #Get the KDA of each team
    table = soup.find_all("table", "table_list trhover")[int(idx)]
    row_marker = 0
    table_data = []

    #lopp for each row in the table
    for row in table.find_all('tr'):
        column_marker = 0
        column_data = []
        columns = row.find_all('td')

        #extract info for each collum and save in the list
        for column in columns:
            column_data.append(column.get_text())
        table_data.append(column_data)

    table_data = table_data[1:]

    df_kda = pd.DataFrame(table_data,columns=['player', 'kda', 'csM', 'dpM','wpM'])
    df_kda.dropna(inplace=True)

    #Apply the function KDA and replace the single values for the metric
    df_kda['kda'] = df_kda.apply(lambda x: player_kda(x['kda']),axis=1)

    return df_kda


### Join data blue team

In [17]:
# dict with blue team data
blue_team_data = {}

team = "blue"

# team name
blue_team_data[f"team_{team}"] = blue_team

# info. if team win or lost
blue_team_data[f"result_{team}"] = blue_result

# bans of team
for idx, ban in enumerate(blue_banned):
    blue_team_data[f"ban_{idx}_{team}"] = ban

# picks of team
for idx, pick in enumerate(blue_piked):
    blue_team_data[f"pick_{idx}_{team}"] = pick

# kills, towers, drgans and barons
blue_team_data.update(get_team_stats(blue_stats, team))

# KDA of each player
player_idx = 0
dct_players = {}
lst_player_kda_keys = ['player', 'kda', 'csM', 'dpM','wpM']

def get_player_kda(row):
    global player_idx
    for idx, i in enumerate(row):
        dct_players[f"{lst_player_kda_keys[idx]}_{player_idx}_{team}"] = i
    player_idx += 1

get_team_kda("blue").apply(lambda x: get_player_kda(x), axis=1)
blue_team_data.update(dct_players)

blue_team_data

{'team_blue': 'Fluxo',
 'result_blue': 'LOSS',
 'ban_0_blue': 'Jayce',
 'ban_1_blue': 'Maokai',
 'ban_2_blue': 'Ashe',
 'ban_3_blue': 'Renekton',
 'ban_4_blue': 'Viktor',
 'pick_0_blue': 'Zac',
 'pick_1_blue': 'Wukong',
 'pick_2_blue': 'Akali',
 'pick_3_blue': 'Zeri',
 'pick_4_blue': 'Lulu',
 'kills_blue': 11,
 'towers_blue': 4,
 'dragons_blue': 3,
 'barons_blue': 0,
 'player_0_blue': 'Tay',
 'kda_0_blue': 1.67,
 'csM_0_blue': '6.1',
 'dpM_0_blue': '378',
 'wpM_0_blue': '0.28',
 'player_1_blue': 'Disamis',
 'kda_1_blue': 4.5,
 'csM_1_blue': '5.1',
 'dpM_1_blue': '381',
 'wpM_1_blue': '0.31',
 'player_2_blue': 'Hauz',
 'kda_2_blue': 2.0,
 'csM_2_blue': '7.4',
 'dpM_2_blue': '532',
 'wpM_2_blue': '0.28',
 'player_3_blue': 'Brance',
 'kda_3_blue': 1.33,
 'csM_3_blue': '9.6',
 'dpM_3_blue': '937',
 'wpM_3_blue': '0.34',
 'player_4_blue': 'JoJo',
 'kda_4_blue': 3.0,
 'csM_4_blue': '0.5',
 'dpM_4_blue': '197',
 'wpM_4_blue': '1.59'}

### Join data Red Team

In [18]:
# dict with red team data
red_team_data = {}

team = "red"

# team name
red_team_data[f"team_{team}"] = red_team

# info. if team win or lost
red_team_data[f"result_{team}"] = red_result

# bans of team
for idx, ban in enumerate(red_banned):
    red_team_data[f"ban_{idx}_{team}"] = ban

# picks of team
for idx, pick in enumerate(red_piked):
    red_team_data[f"pick_{idx}_{team}"] = pick

# kills, towers, drgans and barons
red_team_data.update(get_team_stats(red_stats, team))

# KDA of each player
player_idx = 0
dct_players = {}
lst_player_kda_keys = ['player', 'kda', 'csM', 'dpM','wpM']

def get_player_kda(row):
    global player_idx
    for idx, i in enumerate(row):
        dct_players[f"{lst_player_kda_keys[idx]}_{player_idx}_{team}"] = i
    player_idx += 1

get_team_kda("red").apply(lambda x: get_player_kda(x), axis=1)
red_team_data.update(dct_players)

red_team_data

{'team_red': 'KaBuM! e-Sports',
 'result_red': 'WIN',
 'ban_0_red': 'Ryze',
 'ban_1_red': 'Lucian',
 'ban_2_red': 'Azir',
 'ban_3_red': 'KSante',
 'ban_4_red': 'Kassadin',
 'pick_0_red': 'Jax',
 'pick_1_red': 'Sejuani',
 'pick_2_red': 'TwistedFate',
 'pick_3_red': 'Sivir',
 'pick_4_red': 'Yuumi',
 'kills_red': 19,
 'towers_red': 10,
 'dragons_red': 2,
 'barons_red': 1,
 'player_0_red': 'Lonely',
 'kda_0_red': 7.0,
 'csM_0_red': '7.0',
 'dpM_0_red': '779',
 'wpM_0_red': '0.57',
 'player_1_red': 'scary',
 'kda_1_red': 15.0,
 'csM_1_red': '4.9',
 'dpM_1_red': '383',
 'wpM_1_red': '0.26',
 'player_2_red': 'Yuri',
 'kda_2_red': 4.0,
 'csM_2_red': '7.3',
 'dpM_2_red': '631',
 'wpM_2_red': '0.23',
 'player_3_red': 'DudsTheBoy',
 'kda_3_red': 3.5,
 'csM_3_red': '9.5',
 'dpM_3_red': '866',
 'wpM_3_red': '0.43',
 'player_4_red': 'Scuro',
 'kda_4_red': 16.0,
 'csM_4_red': '0.1',
 'dpM_4_red': '279',
 'wpM_4_red': '1.54'}

### Match all data

In [19]:
dct_teams = {**red_team_data, **blue_team_data}
dct_teams["match_duration"] = match_time

In [20]:
# Dataframe with columns of data in dict
df_match = pd.DataFrame(columns=dct_teams.keys())

In [21]:
# Concat the match data to df with all data
df_match = pd.concat([df_match, pd.DataFrame(dct_teams, index=[0])])

In [22]:
df_match

,team_red,result_red,ban_0_red,ban_1_red,ban_2_red,ban_3_red,ban_4_red,pick_0_red,pick_1_red,pick_2_red,...,kda_3_blue,csM_3_blue,dpM_3_blue,wpM_3_blue,player_4_blue,kda_4_blue,csM_4_blue,dpM_4_blue,wpM_4_blue,match_duration
0,KaBuM! e-Sports,WIN,Ryze,Lucian,Azir,KSante,Kassadin,Jax,Sejuani,TwistedFate,...,1.33,9.6,937,0.34,JoJo,3.0,0.5,197,1.59,00:35:09


In [23]:
# Save .csv with data of all match
df_match.to_csv("data/matchs.csv")